In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import xgboost as xgb
import tensorflow as tf
from gc import collect
import pandas as pd
import os
checkpoint_path = "training_ensemble/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


In [15]:
df_data=pd.read_csv("DataSetNormalised.csv")
df_label=pd.read_csv("LabelSet.csv")

In [16]:
x=np.array(df_data.iloc[:,1:])
y=np.array(df_label.iloc[:,1])
x=x.astype(np.float32)
x_test=x[10000000:11000000][:]
y_test=y[10000000:11000000][:]
x_train=x[:1000000][:]
y_train=y[:1000000][:]

In [10]:
del df_data
del df_label
del x
del y
collect()

0

In [17]:
model_xgb=XGBClassifier()
model_xgb.load_model("XGBoost.json")
y_test_pred_xgb=model_xgb.predict_proba(x_test)
y_train_pred_xgb=model_xgb.predict_proba(x_train)
y_pred_xgb=y_test_pred_xgb[:,1]
y_pred_xgb_train=y_train_pred_xgb[:,1]

In [18]:
y_test_pred_xgb=np.reshape(y_pred_xgb,(1000000,1))
y_train_pred_xgb=np.reshape(y_pred_xgb_train,(1000000,1))
y_train_pred_xgb.shape

(1000000, 1)

In [19]:
model_DNN_5 = tf.keras.models.load_model('DNN_5')
y_test_pred_DNN_5=model_DNN_5.predict(x_test)
y_train_pred_DNN_5=model_DNN_5(x_train)

In [20]:
model_NN_Shallow_1024 = tf.keras.models.load_model('NN_Shallow_1024')
y_test_pred_NN_Shallow_1024=model_NN_Shallow_1024.predict(x_test)
y_train_pred_NN_Shallow_1024=model_NN_Shallow_1024.predict(x_train)

In [21]:
model_NN_Shallow_2048 = tf.keras.models.load_model('NN_Shallow_2048')
y_test_pred_NN_Shallow_2048=model_NN_Shallow_2048.predict(x_test)
y_train_pred_NN_Shallow_2048=model_NN_Shallow_2048.predict(x_train)

In [22]:
model_NN_Shallow_10240 = tf.keras.models.load_model('NN_Shallow_10240')
y_test_pred_NN_Shallow_10240=model_NN_Shallow_10240.predict(x_test)
y_train_pred_NN_Shallow_10240=model_NN_Shallow_10240.predict(x_train)

In [23]:
model_ResNet_52 = tf.keras.models.load_model('ResNet_52')
y_test_pred_model_ResNet_52=model_ResNet_52.predict(x_test)
y_train_pred_model_ResNet_52=model_ResNet_52.predict(x_train)

In [63]:
df_entrain=pd.DataFrame(x_train)
df_entrain['XGB']=y_train_pred_xgb
df_entrain['DNN5']=y_train_pred_DNN_5
df_entrain['S1024']=y_train_pred_NN_Shallow_1024
df_entrain['S2048']=y_train_pred_NN_Shallow_2048
df_entrain['10240']=y_train_pred_NN_Shallow_10240
df_entrain['RN52']=y_train_pred_model_ResNet_52
df_entrain.to_csv("Train_ens.csv")

In [64]:
df_entest=pd.DataFrame(x_test)
df_entest['XGB']=y_test_pred_xgb
df_entest['DNN5']=y_test_pred_DNN_5
df_entest['S1024']=y_test_pred_NN_Shallow_1024
df_entest['S2048']=y_test_pred_NN_Shallow_2048
df_entest['10240']=y_test_pred_NN_Shallow_10240
df_entest['RN52']=y_test_pred_model_ResNet_52
df_entest.to_csv("Test_ens.csv")

In [45]:
x_test_en=np.array(df_entest.iloc[:,28:])

x_train_en=np.array(df_entrain.iloc[:,28:])
x_test_en.shape

(1000000, 6)

In [56]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)


In [58]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(1,input_shape=(6,),activation='sigmoid')
])

In [61]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.01,
        beta_1=0.9,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [62]:


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    x_train_en,
    y_train,
    epochs=100,
    batch_size=10240,
    validation_data=(x_test_en,y_test),
    callbacks=[cp_callback]
)

Epoch 1/100
86/98 [=========================>....] - ETA: 0s - loss: 0.6826 - accuracy: 0.5029 - auc: 0.6433
Epoch 00001: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 8ms/step - loss: 0.6709 - accuracy: 0.5260 - auc: 0.6711 - val_loss: 0.5828 - val_accuracy: 0.7042 - val_auc: 0.8504
Epoch 2/100
86/98 [=========================>....] - ETA: 0s - loss: 0.5321 - accuracy: 0.7606 - auc: 0.8766
Epoch 00002: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.5276 - accuracy: 0.7637 - auc: 0.8769 - val_loss: 0.5050 - val_accuracy: 0.7731 - val_auc: 0.8675
Epoch 3/100
87/98 [=========================>....] - ETA: 0s - loss: 0.4682 - accuracy: 0.7978 - auc: 0.8889
Epoch 00003: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.4660 - accuracy: 0.7986 - auc: 0.8892 - val_loss: 0.4699 - val_accuracy: 0.7847 - val_auc: 0.8720
Epoch 4/100
87/98 [======

89/98 [==========================>...] - ETA: 0s - loss: 0.3617 - accuracy: 0.8401 - auc: 0.9203
Epoch 00026: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3613 - accuracy: 0.8403 - auc: 0.9205 - val_loss: 0.4705 - val_accuracy: 0.7828 - val_auc: 0.8656
Epoch 27/100
90/98 [==========================>...] - ETA: 0s - loss: 0.3609 - accuracy: 0.8406 - auc: 0.9206
Epoch 00027: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 6ms/step - loss: 0.3609 - accuracy: 0.8406 - auc: 0.9206 - val_loss: 0.4723 - val_accuracy: 0.7823 - val_auc: 0.8649
Epoch 28/100
88/98 [=========================>....] - ETA: 0s - loss: 0.3604 - accuracy: 0.8409 - auc: 0.9209
Epoch 00028: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3605 - accuracy: 0.8407 - auc: 0.9208 - val_loss: 0.4740 - val_accuracy: 0.7818 - val_auc: 0.8642
Epoch 29/100
86/98 [===============

89/98 [==========================>...] - ETA: 0s - loss: 0.3588 - accuracy: 0.8420 - auc: 0.9214
Epoch 00051: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213 - val_loss: 0.4898 - val_accuracy: 0.7776 - val_auc: 0.8587
Epoch 52/100
86/98 [=========================>....] - ETA: 0s - loss: 0.3589 - accuracy: 0.8420 - auc: 0.9213
Epoch 00052: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213 - val_loss: 0.4898 - val_accuracy: 0.7775 - val_auc: 0.8586
Epoch 53/100
88/98 [=========================>....] - ETA: 0s - loss: 0.3588 - accuracy: 0.8420 - auc: 0.9213
Epoch 00053: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213 - val_loss: 0.4897 - val_accuracy: 0.7774 - val_auc: 0.8585
Epoch 54/100
87/98 [===============

88/98 [=========================>....] - ETA: 0s - loss: 0.3592 - accuracy: 0.8418 - auc: 0.9212
Epoch 00076: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213 - val_loss: 0.4900 - val_accuracy: 0.7775 - val_auc: 0.8585
Epoch 77/100
97/98 [============================>.] - ETA: 0s - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213
Epoch 00077: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3590 - accuracy: 0.8420 - auc: 0.9213 - val_loss: 0.4900 - val_accuracy: 0.7773 - val_auc: 0.8583
Epoch 78/100
88/98 [=========================>....] - ETA: 0s - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213
Epoch 00078: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 1s 7ms/step - loss: 0.3590 - accuracy: 0.8419 - auc: 0.9213 - val_loss: 0.4900 - val_accuracy: 0.7774 - val_auc: 0.8584
Epoch 79/100
88/98 [===============

In [116]:
y_pred_ensemble=y_test_pred_DNN_5
y_pred_ensemble=y_pred_ensemble+y_test_pred_model_ResNet_52
y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_1024
y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_10240
y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_2048
y_pred_ensemble=y_pred_ensemble+y_test_pred_xgb
y_pred_ensemble=y_pred_ensemble/5
y_pred_ensemble.shape
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ensemble)
metrics.auc(fpr, tpr)
#print("The training accuracy",str(accuracy_score(y_test, np.round(y_pred_ensemble))))

0.8759870002247364

In [24]:
#Trying out average ensemble
y_pred_ensemble=y_test_pred_DNN_5+0*y_test_pred_NN_Shallow_1024
y_pred_ensemble=np.round(y_pred_ensemble)

accuracy = accuracy_score(y_test, y_pred_ensemble)
print("The training accuracy",str(accuracy))
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ensemble)
metrics.auc(fpr, tpr)


The training accuracy 0.788488


0.7865833286210102

In [123]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()